In [1]:
# data wrangling
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
all_data = pd.read_csv("../geotracker/data/all_data.csv").iloc[:, 1:]

In [3]:
all_data

,restaurant_name,avg_review_score,reviews,type_of_cuisine,address,street,zip_code,city_name,coordinates,latitude,longitude,database,opening_hours
0,BunUp Mitte,7.8,NaN,vegetarian or vegan,"This is a virtual venue, 10178.0, Berlin",This is a virtual venue,10178.0,Berlin,"52.5238316,13.3985704",52.523832,13.398570,wolt,NaN
1,Machiavelli,8.4,NaN,european,"Albrechtstr.13 Berlin, 10117.0, Berlin",Albrechtstr.13 Berlin,10117.0,Berlin,"52.52130280012305,13.385180090564926",52.521303,13.385180,wolt,NaN
2,Pizza Peppino,NaN,NaN,european,"Tucholskystraße 13, 10117.0, Berlin",Tucholskystraße 13,10117.0,Berlin,"52.52339332586772,13.391764043640274",52.523393,13.391764,wolt,NaN
3,Fresh's,7.8,NaN,poke,"This is a virtual venue, 10178.0, Berlin",This is a virtual venue,10178.0,Berlin,"52.5238316,13.3985704",52.523832,13.398570,wolt,NaN
4,Bombay,8.4,NaN,middle eastern,"Friedrichstraße 106c, 10117.0, Berlin",Friedrichstraße 106c,10117.0,Berlin,"52.5232743,13.3884602",52.523274,13.388460,wolt,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9254,Cafe Seestern,NaN,NaN,european,Am Schlachtensee 70 Nikolassee 14129 Berlin,Am Schlachtensee 70,14129.0,Nikolassee,"52.44011, 13.21277",52.440110,13.212770,here_maps,NaN
9255,Gabana Restaurant,NaN,NaN,international,Breisgauer Straße 12 Zehlendorf 14129 Berlin,Breisgauer Straße 12,14129.0,Zehlendorf,"52.43783, 13.2154",52.437830,13.215400,here_maps,NaN
9256,Osteria Carlo,NaN,NaN,european,Breisgauer Straße 14 Zehlendorf 14129 Berlin,Breisgauer Straße 14,14129.0,Zehlendorf,"52.43762, 13.2155",52.437620,13.215500,here_maps,NaN
9257,Pizza Piazza,NaN,NaN,NaN,Bülowstraße 1 Zehlendorf 14163 Berlin,Bülowstraße 1,14163.0,Zehlendorf,"52.43707, 13.23353",52.437070,13.233530,here_maps,NaN
